## 获取url对应的标题和单词统计

In [1]:
import feedparser
import re

def get_word_counts(url):
    d = feedparser.parse(url)
    wc = {}

    # 所有文章
    for e in d.entries:
        if 'summary' in e: summary = e.summary
        else: summary = e.description

        # 提取文章中的词
        words = get_words(e.title + ' ' + summary)
        for word in words:
            wc.setdefault(word, 0)
            wc[word] += 1
    return d.feed.title, wc

In [2]:
def get_words(html):
    # 移除html标签
    txt = re.compile(r'<[^>]+>').sub('', html)
    # 利用非字母字符拆分单词
    words = re.compile(r'[^A-Z^a-z]+').split(txt)
    
    return [word.lower() for word in words]

## 对订阅列表中的所有链接进行单词统计

- 由于gfw的原因，很多feedlist无法访问

In [ ]:
apcount = {}
wordcounts = {}
feedlist = [line for line in open('feedlist.txt')]
for feedurl in feedlist:
    try:
        title, wc = get_word_counts(feedurl)
        wordcounts[title] = wc
        for word, count in wc.items():
            apcount.setdefault(word, 0)
            if count > 1:
                apcount[word] += 1
        print('%s' % feedurl)
    except:
        print('Failed to parse feed %s' % feedurl)

Failed to parse feed http://feeds.feedburner.com/37signals/beMH

Failed to parse feed http://feeds.feedburner.com/blogspot/bRuz

Failed to parse feed http://battellemedia.com/index.xml

Failed to parse feed http://blog.guykawasaki.com/index.rdf

http://blog.outer-court.com/rss.xml

Failed to parse feed http://feeds.searchenginewatch.com/sewblog

Failed to parse feed http://blog.topix.net/index.rdf

Failed to parse feed http://blogs.abcnews.com/theblotter/index.rdf

Failed to parse feed http://feeds.feedburner.com/ConsumingExperienceFull

Failed to parse feed http://flagrantdisregard.com/index.php/feed/

Failed to parse feed http://featured.gigaom.com/feed/

Failed to parse feed http://gizmodo.com/index.xml

Failed to parse feed http://gofugyourself.typepad.com/go_fug_yourself/index.rdf

Failed to parse feed http://googleblog.blogspot.com/rss.xml

Failed to parse feed http://feeds.feedburner.com/GoogleOperatingSystem

Failed to parse feed http://headrush.typepad.com/creating_passionate_

### 只有出现频率在10%~50%的单词会被记录

In [ ]:
wordlist = []
for w, bc in apcount.items():
    frac = float(bc) / len(feedlist)
    if frac > 0.1 and frac < 0.5:
        wordlist.append(w)

## 将每篇订阅的词频向量合并为词频矩阵

In [ ]:
out = open('blogdata.csv','w')
# 第一行
out.write('Blog')
for word in wordlist: out.write(',%s' % word)
out.write('\n')
# 后面每行第一个为blog名称，后面为词频
for blog, wc in wordcounts.items():
    print(blog)
    out.write(blog)
    for word in wordlist:
        if word in wc: out.write(',%d' % wc[word])
        else: out.write(',0')
    out.write('\n')
out.close()

## 分层聚类

### 载入数据

In [ ]:
import pandas as pd
import numpy as np

def load_data(path='./blogdata.csv'):
    data = pd.read_csv(path)
    titles = np.array(data['Blog'])
    # 返回numpy数组
    return titles, np.array(data)[:, 1:]

feed_titles, data = load_data()
print(data.shape, feed_titles)

### 定义距离：1-皮尔森相关系数

In [ ]:
from math import sqrt
def pearson_distance(e1, e2):
    sum1 = sum(e1)
    sum2 = sum(e2)
    
    sum_of_square1 = sum([pow(w, 2) for w in e1])
    sum_of_square2 = sum([pow(w, 2) for w in e2])
    
    n = len(e1)
    sum12 = sum([e1[i] *e2[i] for i in range(n)])
    
    num = sum12 - (sum1 * sum2 / n)
    den = sqrt((sum_of_square1 - pow(sum1, 2) / n) * (sum_of_square2 - pow(sum2, 2) / n))
    
    if den == 0: return 0
    return 1 - num / den

In [ ]:
print(pearson_distance(data[3], data[1]))

### 创建聚类class

In [ ]:
class bicluster:
    def __init__(self, vec, left=None, right=None, distance=0, id=None):
        self.vec = vec
        self.left = left
        self.right = right
        self.distance = distance
        self.id = id

### 两两类别合并，构造成树形结构

In [ ]:
def hcluster(data, distance=pearson_distance):
    dis = {}
    current_id = -1
    # 初始化：每个点各成一类
    clust = [bicluster(data[i], id=i) for i in range(data.shape[0])]
    
    while len(clust) > 1:
        
        # 寻找最近的两个类
        closest_pair = (0, 1)
        closest_dis = distance(clust[0].vec, clust[1].vec)
        
        for i in range(len(clust)):
            for j in range(i + 1, len(clust)):
                if (clust[i].id, clust[j].id) not in dis:
                    tmp_dis = distance(clust[i].vec, clust[j].vec)
                if tmp_dis < closest_dis:
                    closest_pair = (i, j)
                    closest_dis = tmp_dis
        
        x, y = closest_pair[0], closest_pair[1]
        mid_vec = [(clust[x].vec[i] + clust[y].vec[i]) / 2 for i in range(data.shape[1])]
        # 两个类合并成新类
        newcluster = bicluster(mid_vec, left=clust[x], right=clust[y], distance=closest_dis, id=current_id)
        clust.append(newcluster)
        current_id -= 1
        
        clustx = clust[x]
        clusty = clust[y]
        # 删除合并前的两个类
        clust.remove(clustx)
        clust.remove(clusty)
        # print(x, y)
        # ls = [clust[i].id for i in range(len(clust))]
        # print(ls)
    # 返回聚类的树根
    return clust[0]

root = hcluster(data)

### 输出聚类树形结构

In [ ]:
def print_tree(root, depth=0):
    for i in range(3 * depth): print(' ', end="")
    
    if root.id < 0: print('--')
    else: print(feed_titles[root.id])
    
    if root.left is not None: print_tree(root.left, depth + 1)
    if root.right is not None: print_tree(root.right, depth + 1)

print_tree(root)

## k-means

In [ ]:
import random

def kcluster(data, distance=pearson_distance, k=3):
    # 在向量空间中随机k个中心点
    vec_range = [(min([data[i][j] for i in range(data.shape[0])]), max([data[i][j] for i in range(data.shape[0])])) 
                 for j in range(data.shape[1])]
    centers = []
    random.seed()
    for i in range(k):
        center = [random.random() * (vec_range[j][1] - vec_range[j][0]) + vec_range[j][0] for j in range(data.shape[1])]
        centers.append(center)
    # 初始化点的类别标签
    labels = {}
    for i in range(data.shape[0]):
        labels[i] = -1
    #print(data)
    while 1:
        # 标记是否有中心变化
        fg = False
        # 计算每个点所属分类
        for i in range(data.shape[0]):
            min_dis = distance(centers[0], data[i])
            min_num = 0
            for j in range(1, k):
                tmp_dis = distance(centers[j], data[i])
                if tmp_dis < min_dis:
                    min_dis = tmp_dis
                    min_num = j
            if labels[i] != min_num:
                fg = True
                labels[i] = min_num
        # 更新中心点
        if fg == False: break
        
        sumv = [[0] * data.shape[1]] * k
        nums = [0] * k
        for i in range(data.shape[0]):
            nums[labels[i]] += 1
            for j in range(data.shape[1]):
                sumv[labels[i]][j] += data[i][j]
        
        for i in range(k):
            if nums[i] != 0:
                for j in range(data.shape[1]):
                    centers[i][j] = sumv[i][j] / nums[i]
        print('sumv: ', sumv)
        print('centers: ', centers)
    return labels

res = kcluster(data)
print(res)